In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import tensorflow as tf  
import tensorflow_federated as tff
from PIL import Image
import nest_asyncio
nest_asyncio.apply()

In [2]:
# Define the path to the dataset
dataset_path = 'UTKFace' 

# Initialize lists to hold images and labels
images = []
ages = []
genders = []
races = []

# Load images and extract labels from filenames
for img_name in os.listdir(dataset_path):
    if img_name.endswith('.jpg'):
        parts = img_name.split('_')
        if len(parts) >= 4:
            try:
                age, gender, race = parts[:3]
                ages.append(int(age))
                genders.append(int(gender))
                races.append(int(race))

                img_path = os.path.join(dataset_path, img_name)
                img = Image.open(img_path).resize((32, 32))
                img = np.array(img)
                images.append(img)
            except ValueError as e:
                print(f"Skipping file {img_name}: {e}")
        else:
            print(f"Skipping file {img_name}: not enough values to unpack (expected 4, got {len(parts)})")

# Convert lists to numpy arrays
images = np.array(images)
ages = np.array(ages)
genders = np.array(genders)
races = np.array(races)

# Encode race labels
label_encoder = LabelEncoder()
y_data_race_encoded = label_encoder.fit_transform(races)

# Normalize the images
images = images.astype(np.float32) / 255.0

Skipping file 39_1_20170116174525125.jpg.chip.jpg: not enough values to unpack (expected 4, got 3)
Skipping file 61_1_20170109142408075.jpg.chip.jpg: not enough values to unpack (expected 4, got 3)
Skipping file 61_1_20170109150557335.jpg.chip.jpg: not enough values to unpack (expected 4, got 3)


In [3]:
# Create a DataFrame to facilitate balancing
data = {
    'image': list(images),
    'age': ages,
    'gender': genders,
    'race': y_data_race_encoded
}
df = pd.DataFrame(data)

# Balance the dataset by oversampling the minority classes
df_majority = df[df.race == df.race.mode()[0]]
df_minority = df[df.race != df.race.mode()[0]]

# Oversample the minority classes
df_minority_oversampled = resample(df_minority,
                                   replace=True,  # Sample with replacement
                                   n_samples=len(df_majority),  # Match number of majority class
                                   random_state=42)  # Reproducible results

# Combine majority class with oversampled minority classes
data_balanced = pd.concat([df_majority, df_minority_oversampled])

# Shuffle the dataset
data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract balanced images and labels
images_balanced = np.array(list(data_balanced['image']))
ages_balanced = np.array(data_balanced['age'])
genders_balanced = np.array(data_balanced['gender'])
races_balanced = np.array(data_balanced['race'])

In [4]:
# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images_balanced, races_balanced, test_size=0.2, random_state=42)

In [5]:
# Create federated data for training
def create_federated_data(data, labels, num_clients=10):
    data_size = len(data)
    client_data = []
    for i in range(num_clients):
        start_idx = i * data_size // num_clients
        end_idx = (i + 1) * data_size // num_clients
        client_data.append((data[start_idx:end_idx], labels[start_idx:end_idx]))
    return client_data

federated_train_data = create_federated_data(x_train, y_train)

def preprocess(dataset):
    def batch_format_fn(image, label):
        image = tf.cast(image, tf.float32)
        return (tf.reshape(image, [32, 32, 3]), tf.reshape(label, [1]))
    return dataset.map(batch_format_fn).batch(20)

federated_train_dataset = [preprocess(tf.data.Dataset.from_tensor_slices((client[0], client[1]))) for client in federated_train_data]

In [6]:
# Define model function
def model_fn():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # Adjust number of classes as needed
    ])
    return tff.learning.from_keras_model(
        model,
       input_spec=(tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32), 
            tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

# Define client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.SGD(learning_rate=0.02)

# Build federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Train the model for a few rounds
for round_num in range(1, 65):
    state, metrics = iterative_process.next(state, federated_train_dataset)
    print(f'Round {round_num}, Metrics={metrics}')

# Prepare test data for federated evaluation
federated_test_data = [preprocess(tf.data.Dataset.from_tensor_slices((x_test, y_test)))]

# Build federated evaluation process
federated_eval = tff.learning.build_federated_evaluation(model_fn)

# Evaluate the model on the federated test data
test_metrics = federated_eval(state.model, federated_test_data)
print(f'Test Metrics: {test_metrics}')

Round 1, Metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.49175143), ('loss', 1.3505528)]))])
Round 2, Metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.5106673), ('loss', 1.3023571)]))])
Round 3, Metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.5308236), ('loss', 1.2607901)]))])
Round 4, Metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.54949147), ('loss', 1.2184476)]))])
Round 5, Metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_

In [7]:
# Extract the trained model weights from the federated learning state
model_weights = state.model.trainable

# Define the Keras model architecture consistently
def create_keras_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')  # Adjust number of classes as needed
    ])
    return model

# Create the Keras model
keras_model = create_keras_model()
keras_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
keras_model.set_weights(model_weights)

# Generate predictions on the test data
y_pred = keras_model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_performance(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    return accuracy, precision, recall, f1

# Evaluate metrics for each race
for race in np.unique(y_test):
    indices = [i for i, r in enumerate(y_test) if r == race]
    y_true_race = y_test[indices]
    y_pred_race = y_pred[indices]
    
    accuracy, precision, recall, f1 = evaluate_performance(y_true_race, y_pred_race)
    print(f'Race: {race}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}\n')

Race: 0
Accuracy: 0.9081, Precision: 0.2000, Recall: 0.1816, F1 Score: 0.1904

Race: 1
Accuracy: 0.8246, Precision: 0.2000, Recall: 0.1649, F1 Score: 0.1808

Race: 2
Accuracy: 0.7365, Precision: 0.2000, Recall: 0.1473, F1 Score: 0.1697

Race: 3
Accuracy: 0.7595, Precision: 0.2000, Recall: 0.1519, F1 Score: 0.1727

Race: 4
Accuracy: 0.1331, Precision: 0.2000, Recall: 0.0266, F1 Score: 0.0470

